In [1]:
import json
import pandas as pd

In [2]:
# function to get shortened wiki articles

N_SENTENCES=3
def get_sentences(x):
    n = N_SENTENCES + 1
    sentences = x.split('. ')
    try:
        return '. '.join(sentences[:n])
    except:
        return '. '.join(sentences)
def count_sentences(x):
    try:
        return len(x.split('. '))
    except:
        return -1

In [3]:
# read pickles

wiki_en = pd.read_pickle('data/wiki_en_summaries.pkl')
wiki_fr = pd.read_pickle('data/wiki_fr_summaries.pkl')
bnf_table = pd.read_pickle('data/bnf_table_summaries.pkl')
bnf_tags = pd.read_pickle('data/bnf_tags.pkl')

named_subjects = pd.read_pickle('data/named_subjects.pkl')

# this line will change 
age_gender = pd.read_json('data/age_gender_labeles.json',orient='records')

In [4]:
# get name-id df (exploded)

named_subjects = named_subjects[['id','title','names']].explode('names')
named_subjects = named_subjects.set_index('id')
named_subjects = named_subjects.rename(columns = {'names':'name'})

# get name-id df (nested in list)

id_img = named_subjects.drop('title', axis=1)
id_img = id_img.reset_index()
id_img = pd.DataFrame(id_img.groupby('name')['id'].apply(list))
id_img = id_img.rename(columns={'id':'id_list'})

In [5]:
# organize dataframes, shorten texts, rename columns

wiki_en['wiki_en_text'] = wiki_en.summary.apply(get_sentences)
wiki_fr['wiki_fr_text'] = wiki_fr.summary_fr.apply(get_sentences)

wiki_en = wiki_en[['name','wiki_en_text','url','weight']]
wiki_fr = wiki_fr[['name','wiki_fr_text','url_fr','weight_fr']]
bnf_summaries = bnf_table[['name','bnf_link','gender','note']]


wiki_en = wiki_en.rename(columns={'url':'wiki_en_link', 'weight':'weight_en'})
wiki_fr = wiki_fr.rename(columns={'url':'wiki_fr_link'})

In [6]:
# link name and id defined in age_gender df

named_age_gender = named_subjects.merge(age_gender[['age','gender','id']], on='id').\
rename(columns={'age':'age_estimate', 'gender':'gender_estimate'})


# if age_gender file is re-writtened, 
#id_name = pd.read_pickle('data/id_name.pkl').rename(columns={'id':'id_list'})
#named_age_gender = id_name.merge(age_gender[['age','gender','id']], on='name').rename(columns={'age':'age_estimate', 'gender':'gender_estimate'})


In [7]:
# merge everything

data_final = pd.merge(bnf_summaries,wiki_en,how='outer')
data_final = pd.merge(data_final,wiki_fr,how='outer')
data_final = pd.merge(data_final, named_age_gender, how='outer')
data_final = data_final.merge(id_img, on='name',how='outer')
data_final = data_final.merge(bnf_tags, on='name',how='outer')


In [8]:
# redefine rank <<<<<< THIS WILL CHANGE 


data_final.weight_en.fillna(-1, inplace=True)
data_final.weight_fr.fillna(-1, inplace=True)

data_final['weight'] = data_final[['weight_en','weight_fr']].mean(axis=1)

In [9]:
data_final = data_final.sort_values(by='weight', ascending=False)

In [10]:
data_final

,name,bnf_link,gender,note,wiki_en_text,wiki_en_link,weight_en,wiki_fr_text,url_fr,weight_fr,id,title,age_estimate,gender_estimate,id_list,tags,weight
235,"Bouguereau, William (1825-1905)",http://data.bnf.fr/ark:/12148/cb12758427r#about,masculin,"Peintre, dessinateur, aquarelliste. - Décorate...",William-Adolphe Bouguereau (French pronunciati...,https://en.wikipedia.org/wiki/William-Adolphe_...,50.0,L'œuvre peint de William Bouguereau (La Rochel...,https://fr.wikipedia.org/wiki/%C5%92uvre_peint...,424.0,https://gallica.bnf.fr/ark:/12148/btv1b53050879q,Bouguereau,43.553167,0.093347,[https://gallica.bnf.fr/ark:/12148/btv1b530508...,"[aquarelliste, décorateur, academiedesbeaxarts...",237.0
521,"Delacroix, Eugène (1798-1863)",http://data.bnf.fr/ark:/12148/cb118991616#about,masculin,"Peintre, aquarelliste, graveur et lithographe",Ferdinand Victor Eugène Delacroix ( DEL-ə-krwa...,https://en.wikipedia.org/wiki/Eug%C3%A8ne_Dela...,56.0,Eugène Delacroix est un peintre français né le...,https://fr.wikipedia.org/wiki/Eug%C3%A8ne_Dela...,83.0,https://gallica.bnf.fr/ark:/12148/btv1b53104547j,Eug Delacroix,45.458278,0.057673,[https://gallica.bnf.fr/ark:/12148/btv1b531045...,"[peintre, graveur, aquarelliste, lithographe, ...",69.5
1573,"Robespierre, Maximilien de (1758-1794)",http://data.bnf.fr/ark:/12148/cb11922216p#about,masculin,Avocat. - Homme politique. - Révolutionnaire,Maximilien François Marie Isidore de Robespier...,https://en.wikipedia.org/wiki/Maximilien_Robes...,43.0,"Maximilien de Robespierre, ou Maximilien Robes...",https://fr.wikipedia.org/wiki/Maximilien_de_Ro...,95.0,https://gallica.bnf.fr/ark:/12148/btv1b53187673w,M Robespierre Reproduction de tableau,38.010203,0.479522,[https://gallica.bnf.fr/ark:/12148/btv1b531876...,"[avocat, hommepolitique, France]",69.0
580,"Dieterle, Amélie (1870-1941)",http://data.bnf.fr/ark:/12148/cb14676815w#about,féminin,Actrice de théâtre et de cinéma. - Cantatrice,NaN,NaN,-1.0,"Amélie Diéterle, nom de scène d'Amélie Laurent...",https://fr.wikipedia.org/wiki/Am%C3%A9lie_Di%C...,129.0,https://gallica.bnf.fr/ark:/12148/btv1b53111808r,Mlle Dieterle Variétés,31.000485,0.719496,[https://gallica.bnf.fr/ark:/12148/btv1b531118...,"[cantatrice, actrice, théâtre, cinéma, France]",64.0
451,"Courbet, Gustave (1819-1877)",http://data.bnf.fr/ark:/12148/cb120519068#about,masculin,"Peintre, sculpteur et graveur. - A été aussi l...","Jean Désiré Gustave Courbet (UK: KOOR-bay, US...",https://en.wikipedia.org/wiki/Gustave_Courbet,61.0,"Gustave Courbet, né le 10 juin 1819 à Ornans e...",https://fr.wikipedia.org/wiki/Gustave_Courbet,64.0,https://gallica.bnf.fr/ark:/12148/btv1b53050548w,"Courbet, peintre",40.302552,0.051735,[https://gallica.bnf.fr/ark:/12148/btv1b530505...,"[sculpteur, artiste, graveur, lithographe, pei...",62.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,"Jambon, Marcel (1848-1908)",http://data.bnf.fr/ark:/12148/cb146529838#about,masculin,Décorateur de théâtre et peintre,NaN,NaN,-1.0,NaN,NaN,-1.0,https://gallica.bnf.fr/ark:/12148/btv1b531542380,Jambon Peintre décorateur,41.558484,0.029530,[https://gallica.bnf.fr/ark:/12148/btv1b531542...,"[peintre, décorateur, théâtre, France]",-1.0
941,"James, Constantin (1813-1888)",http://data.bnf.fr/ark:/12148/cb12334796b#about,masculin,"Docteur en médecine (Paris, 1840). - Chargé d'...",NaN,NaN,-1.0,NaN,NaN,-1.0,https://gallica.bnf.fr/ark:/12148/btv1b530664695,Dr Constantin James,43.988517,0.040297,[https://gallica.bnf.fr/ark:/12148/btv1b530664...,"[docteur, médecine, France]",-1.0
944,"Jannin, Philippe (1845?-19..)",http://data.bnf.fr/ark:/12148/cb170663924#about,masculin,"Neveu du caricaturiste Henry Monnier, il fut c...",NaN,NaN,-1.0,NaN,NaN,-1.0,https://gallica.bnf.fr/ark:/12148/btv1b531330420,Jammin i e Jannin Bouffes Babiole,36.449915,0.118342,[https://gallica.bnf.fr/ark:/12148/btv1b531330...,"[caricaturiste, chanteur, France]",-1.0
946,"Janvier, Elisabeth (1855-19..)",http://data.bnf.fr/ark:/12148/cb155900415#about,féminin,

In [13]:
data_final = data_final[['id','name','gender','id_list', 'weight','note', 'bnf_link', 'wiki_en_text', 'wiki_en_link',
        'wiki_fr_text', 'url_fr', 'age_estimate','gender_estimate','tags']]

In [14]:
data_final.to_pickle('data/merged_dataframe.pkl')